<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8d922d46cdeff2efc0aab5ac057fc2beea471dc78d05c6ccaa2f5b197ded762f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-03 11:44:54
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 28.81 K (0.21%)
Current PnL: -21.02 L (-13.95%)
CY Booked + Current PnL: -9.58 L (-6.36%)
-------------------
Total profit:  1.33 L
Total loss:  -22.36 L
-------------------
Total Booked + Current PnL: 17.40 L (14.06%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.14%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.34 L (62.84%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.8%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,70.0,H-MC,13.05,118095.0,17591.0,11514.0,3.88,17.50,9.75,28.96,79.0,1.53,0.84,57.41,MH,ATH,METALS
77,TTKPRESTIG,770.00,94.88,41.0,M-SC,1.69,83997.0,-16780.0,16875.0,0.65,-16.65,20.09,0.09,245.0,-0.99,0.60,10.35,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,8.41,43.0,H-MC,2.90,187000.0,22330.0,21000.0,-0.42,13.56,11.23,26.31,89.0,1.06,1.34,22.63,X40N,NTT,AC
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197849.0,-2289.0,23188.0,-0.25,-1.14,11.72,10.44,4.0,-0.10,1.41,4.36,X40,NTT,FMCG
34,ICICIGI,2260.25,-18.49,52.0,H-MC,1.74,140326.0,4193.0,26929.0,-0.38,3.08,19.19,22.86,91.0,0.16,1.00,16.08,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-17.45,59.0,H-SC,18.59,221819.0,-48942.0,378889.0,5.00,-18.08,170.81,121.86,133.0,-0.13,1.58,73.52,AR,ATH,IT
12,BATAINDIA,2096.00,-31.44,58.0,M-SC,4.73,102447.0,-26223.0,71518.0,3.89,-20.38,69.81,35.20,219.0,-0.37,0.73,17.59,X40,NTT,FOOTWEAR
50,NATIONALUM,244.55,-41.18,70.0,H-MC,13.05,118095.0,17591.0,11514.0,3.88,17.50,9.75,28.96,79.0,1.53,0.84,57.41,MH,ATH,METALS
7,ATULAUTO,844.00,3613.51,51.0,M-SC,8.21,166668.0,-30464.0,120284.0,3.24,-15.45,72.17,45.57,236.0,-0.25,1.19,18.16,XY24,NTT,AUTO
32,HINDZINC,730.22,32.35,69.0,M-LC,4.51,215260.0,10184.0,102378.0,2.80,4.97,47.56,54.89,52.0,0.10,1.54,30.26,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,70.0,M-SC,9.37,145791.0,-3429.0,147613.0,-4.48,-2.30,101.25,96.62,208.0,-0.02,1.04,57.78,XY25,NTT,FINANCE
64,SHALBY,327.00,1222.17,65.0,M-SC,18.16,179834.0,-1309.0,46775.0,-1.76,-0.72,26.01,25.10,235.0,-0.03,1.28,44.12,XY24,NTT,HEALTHCARE
49,MASFIN,398.61,-20.51,40.0,H-SC,11.54,90480.0,-7500.0,29107.0,-1.71,-7.65,32.17,22.05,152.0,-0.26,0.65,31.52,XR,ATH,FINANCE
66,SIS,528.00,2070.98,40.0,H-SC,3.70,87618.0,-23414.0,47025.0,-1.58,-21.09,53.67,21.26,156.0,-0.50,0.63,18.38,OX40N,NTT,MISC
20,COALINDIA,497.97,5.97,42.0,L-LC,4.23,141802.0,-8658.0,42441.0,-1.35,-5.75,29.93,22.46,167.0,-0.20,1.01,11.28,XY25,ATH,MINING


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1222.17,65.0,M-SC,18.16,179834.0,-1309.0,46775.0,-1.76,-0.72,26.01,25.10,235.0,-0.03,1.28,44.12,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.0,-21.68,44.0,H-MC,2.20,134685.0,-1103.0,43853.0,-0.11,-0.81,32.56,31.48,107.0,-0.03,0.96,11.21,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,64.0,H-SC,12.07,129681.0,-12798.0,32317.0,-0.41,-8.98,24.92,13.70,163.0,-0.40,0.93,53.06,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.06,61.0,H-SC,1.86,227358.0,-42309.0,78643.0,1.36,-15.69,34.59,13.47,138.0,-0.54,1.62,14.99,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,94.88,41.0,M-SC,1.69,83997.0,-16780.0,16875.0,0.65,-16.65,20.09,0.09,245.0,-0.99,0.60,10.35,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.43,40.0,H-SC,1.48,141496.0,-34407.0,76422.0,0.75,-19.56,54.01,23.89,149.0,-0.45,1.01,22.20,OX40N,NTT,CERAMICS
66,SIS,528.0,2070.98,40.0,H-SC,3.70,87618.0,-23414.0,47025.0,-1.58,-21.09,53.67,21.26,156.0,-0.50,0.63,18.38,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.96,64.0,M-MC,11.99,236465.0,11503.0,156161.0,1.78,5.11,66.04,74.53,192.0,0.07,1.69,39.02,XY24,BTT,STEEL
32,HINDZINC,730.22,32.35,69.0,M-LC,4.51,215260.0,10184.0,102378.0,2.80,4.97,47.56,54.89,52.0,0.10,1.54,30.26,X5K,ATH,METALS
14,BLUESTARCO,2080.00,8.41,43.0,H-MC,2.90,187000.0,22330.0,21000.0,-0.42,13.56,11.23,26.31,89.0,1.06,1.34,22.63,X40N,NTT,AC
84,VOLTAS,1530.00,-3.60,43.0,H-MC,2.19,202395.0,10653.0,27101.0,-0.31,5.56,13.39,19.69,99.0,0.39,1.45,12.73,XY25,NTT,AC
17,CAMS,5211.76,-6.05,45.0,H-SC,1.98,106789.0,4785.0,39138.0,-0.24,4.69,36.65,43.06,122.0,0.12,0.76,22.61,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-41.18,70.0,H-MC,13.05,118095.0,17591.0,11514.0,3.88,17.50,9.75,28.96,79.0,1.53,0.84,57.41,MH,ATH,METALS
11,BANKINDIA,190.00,-26.71,65.0,H-MC,9.99,190558.0,10750.0,101663.0,-0.59,5.98,53.35,62.52,88.0,0.11,1.36,42.10,XR,NTT,BANKS
32,HINDZINC,730.22,32.35,69.0,M-LC,4.51,215260.0,10184.0,102378.0,2.80,4.97,47.56,54.89,52.0,0.10,1.54,30.26,X5K,ATH,METALS
37,INDIAMART,4850.92,-54.45,37.0,H-SC,9.41,127237.0,3901.0,129858.0,0.07,3.16,102.06,108.45,119.0,0.03,0.91,26.22,AR,ATH,MISC
86,WIPRO,420.00,-15.61,39.0,M-LC,5.87,148496.0,-2449.0,111907.0,-0.65,-1.62,75.36,72.51,53.0,-0.02,1.06,4.30,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.29,30.0,H-LC,5.80,206351.0,-45417.0,98636.0,0.39,-18.04,47.80,21.13,27.0,-0.46,1.47,11.00,X40,ATH,PAINTS
83,VBL,671.64,-22.30,30.0,H-LC,9.58,279731.0,-36111.0,148789.0,-1.02,-11.43,53.19,35.67,5.0,-0.24,2.00,0.84,X40N,ATH,FMCG
74,TCS,4476.75,-31.58,31.0,H-LC,12.47,272252.0,-73704.0,148568.0,-0.61,-21.30,54.57,21.64,1.0,-0.50,1.94,0.27,X40,ATH,IT
30,HCLTECH,1943.91,-4.52,34.0,H-LC,10.30,217822.0,-24102.0,87368.0,-0.15,-9.96,40.11,26.15,8.0,-0.28,1.56,2.93,X40,ATH,IT
57,RELIANCE,1533.00,-15.17,38.0,H-LC,3.54,212066.0,1700.0,27081.0,-0.68,0.81,12.77,13.68,37.0,0.06,1.51,17.47,XY25,NTT,REFINERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-2.40,45.0,H-LC,1.77,156775.0,-29320.0,76804.0,0.45,-15.76,48.99,25.51,15.0,-0.38,1.12,15.53,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-6.65,44.0,H-LC,1.82,231458.0,-14693.0,93903.0,0.40,-5.97,40.57,32.18,16.0,-0.16,1.65,27.92,X200,ATH,IT
42,ITC,452.0,-38.77,46.0,H-LC,2.17,197849.0,-2289.0,23188.0,-0.25,-1.14,11.72,10.44,4.0,-0.10,1.41,4.36,X40,NTT,FMCG
57,RELIANCE,1533.0,-15.17,38.0,H-LC,3.54,212066.0,1700.0,27081.0,-0.68,0.81,12.77,13.68,37.0,0.06,1.51,17.47,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.66,44.0,H-LC,3.77,271721.0,6295.0,50486.0,-0.45,2.37,18.58,21.40,11.0,0.12,1.94,9.63,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,31.0,H-LC,12.47,272252.0,-73704.0,148568.0,-0.61,-21.30,54.57,21.64,1.0,-0.50,1.94,0.27,X40,ATH,IT
83,VBL,671.64,-22.30,30.0,H-LC,9.58,279731.0,-36111.0,148789.0,-1.02,-11.43,53.19,35.67,5.0,-0.24,2.00,0.84,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.52,34.0,H-LC,10.30,217822.0,-24102.0,87368.0,-0.15,-9.96,40.11,26.15,8.0,-0.28,1.56,2.93,X40,ATH,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197849.0,-2289.0,23188.0,-0.25,-1.14,11.72,10.44,4.0,-0.10,1.41,4.36,X40,NTT,FMCG
40,INFY,2275.00,-21.03,39.0,H-LC,8.24,307487.0,-5401.0,177082.0,-0.15,-1.73,57.59,54.87,3.0,-0.03,2.20,4.81,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.40,45.0,H-LC,1.77,156775.0,-29320.0,76804.0,0.45,-15.76,48.99,25.51,15.0,-0.38,1.12,15.53,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197849.0,-2289.0,23188.0,-0.25,-1.14,11.72,10.44,4.0,-0.10,1.41,4.36,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.29,30.0,H-LC,5.80,206351.0,-45417.0,98636.0,0.39,-18.04,47.80,21.13,27.0,-0.46,1.47,11.00,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.18,44.0,H-LC,3.91,207829.0,3329.0,38095.0,0.26,1.63,18.33,20.25,10.0,0.09,1.48,10.50,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-15.17,38.0,H-LC,3.54,212066.0,1700.0,27081.0,-0.68,0.81,12.77,13.68,37.0,0.06,1.51,17.47,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,31.0,H-LC,12.47,272252.0,-73704.0,148568.0,-0.61,-21.30,54.57,21.64,1.0,-0.50,1.94,0.27,X40,ATH,IT
40,INFY,2275.00,-21.03,39.0,H-LC,8.24,307487.0,-5401.0,177082.0,-0.15,-1.73,57.59,54.87,3.0,-0.03,2.20,4.81,X40,BTT,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.17,197849.0,-2289.0,23188.0,-0.25,-1.14,11.72,10.44,4.0,-0.10,1.41,4.36,X40,NTT,FMCG
83,VBL,671.64,-22.30,30.0,H-LC,9.58,279731.0,-36111.0,148789.0,-1.02,-11.43,53.19,35.67,5.0,-0.24,2.00,0.84,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,46.0,H-LC,7.65,249336.0,-12283.0,131500.0,-0.09,-4.70,52.74,45.57,7.0,-0.09,1.78,6.09,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,37.50,79872.0,-33677.0,73730.0,0.43,-29.66,92.31,35.27,268.0,-0.46,0.57,95.49,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,40.0,H-SC,11.54,90480.0,-7500.0,29107.0,-1.71,-7.65,32.17,22.05,152.0,-0.26,0.65,31.52,XR,ATH,FINANCE
50,NATIONALUM,244.55,-41.18,70.0,H-MC,13.05,118095.0,17591.0,11514.0,3.88,17.50,9.75,28.96,79.0,1.53,0.84,57.41,MH,ATH,METALS
78,UJJIVANSFB,60.00,105.37,64.0,H-SC,12.07,129681.0,-12798.0,32317.0,-0.41,-8.98,24.92,13.70,163.0,-0.40,0.93,53.06,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,70.0,M-SC,9.37,145791.0,-3429.0,147613.0,-4.48,-2.30,101.25,96.62,208.0,-0.02,1.04,57.78,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1222.17,65.0,M-SC,18.16,179834.0,-1309.0,46775.0,-1.76,-0.72,26.01,25.10,235.0,-0.03,1.28,44.12,XY24,NTT,HEALTHCARE
69,SURYODAY,240.00,73.25,81.0,H-SC,3.91,159946.0,-19125.0,84851.0,0.78,-10.68,53.05,36.71,135.0,-0.23,1.14,58.70,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,70.0,M-SC,9.37,145791.0,-3429.0,147613.0,-4.48,-2.30,101.25,96.62,208.0,-0.02,1.04,57.78,XY25,NTT,FINANCE
71,TANLA,1963.11,-17.45,59.0,H-SC,18.59,221819.0,-48942.0,378889.0,5.00,-18.08,170.81,121.86,133.0,-0.13,1.58,73.52,AR,ATH,IT
7,ATULAUTO,844.00,3613.51,51.0,M-SC,8.21,166668.0,-30464.0,120284.0,3.24,-15.45,72.17,45.57,236.0,-0.25,1.19,18.16,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.96
1,25,43.13
2,50,74.77


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.27
LC    32.00
MC    23.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.92
X40      14.42
X40N     11.22
XR       11.09
XY25     10.33
AR        8.18
OX40N     7.79
MH        1.65
X200      1.65
X5K       1.54
SR        1.16
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.11
H-LC    25.39
H-MC    20.37
M-SC    13.72
M-LC     5.60
M-MC     3.01
L-SC     1.44
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.74,-6.65,41.85
IT,12.51,-19.95,85.35
FINANCE,10.39,-16.05,68.76
MISC,6.93,-18.58,84.51
BANKS,6.42,-10.97,69.35
PAINTS,5.58,-17.97,35.55
ELECTRICAL,5.32,-10.43,49.17
INSURANCE,3.73,-5.74,45.27
AUTO,3.58,-14.75,69.61


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3242405.0,22
XR,1356988.0,14
AR,1185574.0,9
X40,828321.0,10
X40N,634726.0,9
OX40N,557927.0,10
XY25,490287.0,7
SR,258965.0,2
X5K,102378.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3716294.0,29
M-SC,1508315.0,17
H-LC,1263950.0,15
H-MC,1252297.0,15
M-LC,367372.0,4
M-MC,362481.0,2
L-SC,260373.0,3
L-MC,60139.0,1
L-LC,42441.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1227669.0      6
M-SC       XY24       853258.0      7
H-SC       AR         810059.0      5
           XR         782018.0      7
H-MC       XY24       572112.0      4
H-LC       X40        534842.0      5
M-MC       XY24       362481.0      2
H-SC       X40N       311227.0      4
           SR         258965.0      2
           OX40N      255682.0      4
H-LC       X40N       236324.0      3
H-MC       X40        221961.0      4
H-LC       AR         208304.0      2
H-MC       XY25       183454.0      2
L-SC       XR         171435.0      2
M-SC       AR         167211.0      2
M-LC       XY24       153087.0      2
M-SC       XY25       147613.0      1
           OX40N      138889.0      4
           XR         129826.0      2
H-LC       XY25       116779.0      3
M-LC       XR         111907.0      1
           X5K        102378.0      1
H-MC       XR         101663.0      1
H-LC       X200        93903.0      1
L-SC       OX40N       88938.0      1
H-MC       X40N        87175.0      2
           OX40N       74418.0      1
H-LC       XY24        73798.0      1
M-SC       X40         71518.0      1
H-SC       MH          70674.0      1
L-MC       XR          60139.0      1
L-LC       XY25        42441.0      1
H-MC       MH          11514.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
